## Scatter plot for the different drone components

In [1]:
# dominated function. variable input: reference point, dataframe and selection criterium
# function to determine the pareto optimum according to multiple conditions
# return counter=1 if dominated, else 0 for dominant point
# here args[0] is the reference point, args[1] is the dataframe and args[2] is the selection criterium

def dominated(*args):
    counter=0
    for argsDF in zip(*args[1]):# first row of dataframe 
        subcounter=0
        for x1,x2,x3 in zip(args[0],argsDF,args[2]):
            if x3=='max.': #if criterium is to maximize,
                if x2>x1:
                    subcounter+=1
                else: 
                    None
            else:
                if x2<x1:
                    subcounter+=1
                else: 
                    None
            if subcounter==len(args[0]):# if condition is respected n times, the point is dominated, counter !=0
                counter+=1
    return counter

In [2]:
# FUNCTION: rename (create a string to be executed in the code later on). Input: output given in the widget
# the goal of this function is to transform the output given by the widget into a sentence which can be computed in the code
# i.e. if output of widget is ['Thrust', 'max.', 'DIAMETER', 'min.'], we create a string chain as 
# df.Thrust[row],df.DIAMETER[row] and df.Thrust,df.DIAMETER, which will be applied in the loop to calculate the pareto.

def rename(mylist):
    argsV = {} # dict to save the variables with the format of df.Variable[row]
    argsDF = {}# dict to save the variables with the format of df.Variable
    chars = {} # dict to save the criteria : maximize the variable (max.) or (min.)
    counter=0

    for idx,x in enumerate(mylist):

        if idx%2 ==0:
            argsV["x{0}".format(counter)] = 'df.'+mylist[counter]+'[row]'
            argsDF["x{0}".format(counter)] = 'df.'+mylist[counter]
            counter+=1

        else:
            chars["c{0}".format(counter)] = mylist[counter]
            counter+=1

    argsV = list(argsV.values())    
    argsDF= list(argsDF.values())  

    chars=list(chars.values())


    argsV = ','.join(argsV)
    argsDF = ','.join(argsDF)
    display(argsV, argsDF, chars)
    return argsV, argsDF, chars

In [3]:
# FUNCTION: create_download_link
# goal: create a link with the output csv file with to download

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


### Automatic creation of pareto charts for motor, battery, ESC and propellers components

In [4]:
# %matplotlib inline
from ipywidgets import *
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd

import base64
from IPython.display import HTML


In [5]:
## Select the number of selection criteria for each component

NrMotCrits=widgets.BoundedIntText(
    value=2,
    min=2,
    max=5,
    step=0.1,
    description='set of motor criteria :',
    disabled=False,
    style = {'description_width': '100pt'},
    layout = {'width': '150pt'}
)
NrBatCrits=widgets.BoundedIntText(
    value=2,
    min=2,
    max=5,
    step=0.1,
    description='set of battery criteria :',
    disabled=False,
    style = {'description_width': '100pt'},
    layout = {'width': '150pt'}
)
NrESCCrits=widgets.BoundedIntText(
    value=2,
    min=2,
    max=5,
    step=0.1,
    description='set of ESC criteria :',
    disabled=False,
    style = {'description_width': '100pt'},
    layout = {'width': '150pt'}
)

NrPropCrits=widgets.BoundedIntText(
    value=2,
    min=2,
    max=5,
    step=0.1,
    description='set of propeller criteria :',
    disabled=False,
    style = {'description_width': '130pt'},
    layout = {'width': '170pt'}
)

box_layout = Layout(display='flex',
                    flex_flow='row',
                    justify_content='flex-start',
                    border='solid',
                    background_color='grey')

hb = widgets.HBox(layout=box_layout)

hb.children=[NrMotCrits,NrBatCrits,NrESCCrits,NrPropCrits]

display(hb)


In [7]:
# %matplotlib widget
# to plot the scatter matrix

import matplotlib.pyplot as plt
import numpy as np

# download link

import base64
from IPython.display import HTML

# required to obtain the widgets

import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display,clear_output

In [9]:

### Dataframe references.
path='./Motors/'
df_mot = pd.read_csv(path+'Motors_Data.csv', sep=';')

path='./ESC/'
df_esc = pd.read_csv(path+'ESC_data.csv', sep=';')

path='./Batteries/'
df_bat = pd.read_csv(path+'Batteries_Data.csv', sep=';')

path='./Propeller/'
df_pro = pd.read_csv(path+'Propeller_Data.csv', sep=';')

out = widgets.Output()# required to show the different criteria as the number of criteria change


@out.capture()# required to show the different criteria as the number of criteria change



def changing_pars(x): # as number of criteria change, the different component criteria are displayed
    clear_output() 
    
    button = widgets.Button(
        description='Plot',
        disabled=False,
        button_style='info',
        tooltip='Click me',
        icon='check'
    )

    
    sortie = widgets.Output()

    @sortie.capture()


    def on_button_clicked(b):
        
        plt.close("all") # required not to accumulate plots of figures as button is clicked successively
        clear_output() 
        display('loading')     
        
        # output of widgets are displayed as: ['Thrust', 'max.', 'DIAMETER', 'min.',...]
        mylistPP=[]
        mylistPC=[]
        mylistBP=[]
        mylistBC=[]
        mylistEP=[]
        mylistEC=[]
        mylistMP=[]
        mylistMC=[]
        for i,l in enumerate(j):
            mylistPP.append(mylistProPars[i].value)
            mylistPC.append(mylistProCrit[i].value)
        for i,l in enumerate(d):
            mylistMP.append(mylistMotPars[i].value)
            mylistMC.append(mylistMotCrit[i].value)
        for i,l in enumerate(h):
            mylistEP.append(mylistESCPars[i].value)
            mylistEC.append(mylistESCCrit[i].value)
        for i,l in enumerate(f):
            mylistBP.append(mylistBatPars[i].value)
            mylistBC.append(mylistBatCrit[i].value)
            
        resultPro = [item for sublist in zip(mylistPP,mylistPC) for item in sublist] 
        resultBat = [item for sublist in zip(mylistBP,mylistBC) for item in sublist]
        resultESC = [item for sublist in zip(mylistEP,mylistEC) for item in sublist]
        resultMot = [item for sublist in zip(mylistMP,mylistMC) for item in sublist]
        #-------------------------------#
        # PARETO PROPELLER. PLOT OF SCATTER MATRIX AND DOWNLOAD LINK
#         print(resultPro)
        
        outputV,outputdf,chars=rename(resultPro) #transform it to string, to be processable for the code
#         print(outputdf)
        df=df_pro
        df_pro['Dominated']=0
        
        for row in range(len(df_pro['Ct'])):
            if dominated(eval(outputV),eval(outputdf),chars)>0:
                df_pro.loc[row,'Dominated']=1    

        
        color_wheel = {0: 'r', 
                       1: 'b'} #we color in red the non-dominated series

        colorsP = df_pro['Dominated'].map(lambda x: color_wheel.get(x))

        pd.plotting.scatter_matrix(df_pro[['BETA','DIAMETER_IN','Ct','Cp']], color=colorsP, figsize=[15,10], s=200)
        plt.suptitle('Propeller')
        plt.tick_params(axis = 'both', labelsize = 14)
        plt.rcParams.update({'font.size': 14})
        display(create_download_link(df_pro[df_pro['Dominated']==0],title='"Download CSV file pareto propellers', filename='propellers_pareto.csv'))

        #-------------------------------#
        # DISPLAY OF PARETO OPTIMUM FOR BATTERY. PLOT OF SCATTER MATRIX AND DOWNLOAD LINK

#         print(resultBat)
        outputV,outputdf,chars=rename(resultBat)#transform it to string, to be processable for the code
        df=df_bat
        
        df_bat['Dominated']=0

        for row in range(len(df_bat['Weight_g'])):
            if dominated(eval(outputV),eval(outputdf),chars)>0:
                df_bat.loc[row,'Dominated']=1    


        color_wheel = {0: 'r', 
                       1: 'b'}

        colorsB = df_bat['Dominated'].map(lambda x: color_wheel.get(x))

        pd.plotting.scatter_matrix(df_bat[['Energy_J','Capacity_mAh','Weight_g','Voltage_V','Energy_density_Wh_kg']], color=colorsB, figsize=[15,10], s=200);
        plt.suptitle('Battery')
        plt.tick_params(axis = 'both', labelsize = 14)
        plt.rcParams.update({'font.size': 14})
        display(create_download_link(df_bat[df_bat['Dominated']==0],title='"Download CSV file pareto batteries', filename='batteries_pareto.csv'))

        #-------------------------------#
        # DISPLAY OF PARETO OPTIMUM FOR ESC. PLOT OF SCATTER MATRIX AND DOWNLOAD LINK
        
#         print(resultESC)
#         
        path='./ESC/'
        df_esc = pd.read_csv(path+'ESC_data.csv', sep=';')

        outputV,outputdf,chars=rename(resultESC)#transform it to string, to be processable for the code
        df=df_esc
        
        df_esc['Dominated']=0

        for row in range(len(df_esc['Power_max_W'])):
            if dominated(eval(outputV),eval(outputdf),chars)>0:
                df_esc.loc[row,'Dominated']=1    

        color_wheel = {0: 'r', 
                       1: 'b'}

        colorsE = df_esc['Dominated'].map(lambda x: color_wheel.get(x))

        df_esc=df_esc.loc[:, df_esc.columns != 'Dominated']

        pd.plotting.scatter_matrix(df_esc, color=colorsE, figsize=[15,10], s=200)
        plt.suptitle('ESC')
        plt.tick_params(axis = 'both', labelsize = 14)
        plt.rcParams.update({'font.size': 14})    
        display(create_download_link(df_esc,title='"Download CSV file pareto ESC', filename='esc_pareto.csv'))   

        #-------------------------------#
        # DISPLAY OF PARETO OPTIMUM FOR MOTOR. PLOT OF SCATTER MATRIX AND DOWNLOAD LINK
                
#         print(resultMot)
        outputV,outputdf,chars=rename(resultMot)#transform it to string, to be processable for the code
        df=df_mot
        
        df_mot['Dominated']=0

        for row in range(len(df_mot['Tnom_Nm'])):
            if dominated(eval(outputV),eval(outputdf),chars)>0:
                df_mot.loc[row,'Dominated']=1    

        color_wheel = {0: 'r', 
                       1: 'b'}

        colorsM = df_mot['Dominated'].map(lambda x: color_wheel.get(x))

        pd.plotting.scatter_matrix(df_mot[['Tnom_Nm','Mass_g','Kt_Nm_A']], color=colorsM, figsize=[15,10], s=200)

        plt.suptitle('Motor')
        plt.tick_params(axis = 'both', labelsize = 14)
        plt.rcParams.update({'font.size': 14})
        display(create_download_link(df_mot[df_mot['Dominated']==0],title='"Download CSV file pareto Motors', filename='motors_pareto.csv'))
#         print(mylistC)
#         print('hola')
        plt.show()

        #------------#------------#------------#------------#------------#------------#------------#
    # here we display the WIDGETS!
    
    d={}# dictionary to save the dropdown parameter
    e={}# dictionary to save the radio button parameter
    

    for x in range(0,NrMotCrits.value):
        d["string{0}".format(x)]=widgets.Dropdown(
        options= df_mot.columns.values,
        description='Motor parameter',
        value='Tnom_Nm',
        disabled=False,style = {'description_width': '130pt'})

        e["string{0}".format(x)]=widgets.RadioButtons(
        options=['max.', 'min.'],
        description='Motor critery: ',
        style = {'description_width': '130pt'},
        disabled=False
        )

    mylistMotPars=[]
    mylistMotCrit=[]
    for l in d:
        mylistMotPars.append(d[l])
        mylistMotCrit.append(e[l])

    f={}
    g={}
    for x in range(0,NrBatCrits.value):
        f["string{0}".format(x)]=widgets.Dropdown(
        options= df_bat.columns.values,
        value='Energy_J',
        description='Battery parameter:',
        style = {'description_width': '130pt'},
        disabled=False)


        g["string{0}".format(x)]=widgets.RadioButtons(
        options=['max.', 'min.'],
        description='Battery critery: ',
        style = {'description_width': '130pt'},
        disabled=False
        )

    mylistBatPars=[]
    mylistBatCrit=[]
    for l in f:
        mylistBatPars.append(f[l])
        mylistBatCrit.append(g[l])
#         f[l].observe(on_change, names=['value'])
#         g[l].observe(on_change, names=['value'])
# #     return display(HBox(mylistPars),HBox(mylistCrit))

    h={}
    i={}
    for x in range(0,NrESCCrits.value):
        h["string{0}".format(x)]=widgets.Dropdown(
        options= df_esc.columns.values,
        value='Power_max_W',
        description='ESC parameter:',
        style = {'description_width': '130pt'},
        disabled=False)

        i["string{0}".format(x)]=widgets.RadioButtons(
        options=['max.', 'min.'],
        description='ESC critery: ',
        style = {'description_width': '130pt'},
        disabled=False
        )

    mylistESCPars=[]
    mylistESCCrit=[]
    for l in h:
        mylistESCPars.append(h[l])
        mylistESCCrit.append(i[l])
#         h[l].observe(on_change, names=['value'])
#         i[l].observe(on_change, names=['value'])
#     return display(HBox(mylistESCPars),HBox(mylistESCCrit))

    j={}
    k={}
    
    for x in range(0,NrPropCrits.value):
        j["string{0}".format(x)]=widgets.Dropdown(
        options= df_pro.columns.values,
        value='Thrust_Lbf',
        description='Propeller parameter:',
        style = {'description_width': '130pt'},
        disabled=False)

        k["string{0}".format(x)]=widgets.RadioButtons(
        options=['max.', 'min.'],
        description='Propeller critery: ',
        style = {'description_width': '130pt'},
        disabled=False
        )
    mylistProPars=[]
    mylistProCrit=[]

    for l in j:
        mylistProPars.append(j[l])
        mylistProCrit.append(k[l])
    
        
    vb=VBox([HBox(mylistMotPars),HBox(mylistMotCrit),HBox(mylistBatPars),HBox(mylistBatCrit),HBox(mylistESCPars),HBox(mylistESCCrit),HBox(mylistProPars),HBox(mylistProCrit)])
    button.on_click(on_button_clicked)
    
#     print(result)
    return display(vb), display(button), display(sortie)

changing_pars(NrMotCrits)#first display of widgets
changing_pars(NrBatCrits)#first display of widgets
changing_pars(NrESCCrits)#first display of widgets
changing_pars(NrPropCrits)#first display of widgets


NrMotCrits.observe(changing_pars)#feedback as number of motor criteries change
NrBatCrits.observe(changing_pars)#feedback as number of battery criteries change
NrESCCrits.observe(changing_pars)#feedback as number of ESC criteries change
NrPropCrits.observe(changing_pars)#feedback as number of prop criteries change

display(out)# printing pareto as button is clicked

Output()